# What to minimize ?
The way we measuring risk is by <b>standard deviation</b>. However, 
- <b>Variance</b> is preferred rather than <b>standard deviation</b>, since it gives us a <b>quadratic form</b>, which is easy to differentiate.
<br>
- Square root is <b>monotonically increasing</b> function, so minimizing <b>variance</b> also minimize <b>standard deviation</b>.

-----
<b>Quadratic form</b>: Polynomial with terms all of degree two.
<br>
<b>Monotonically increasing</b>: Always increasing; never remaining constant or decreasing.
<br>
<br>
<br>
σ<sub>p</sub><sup>2</sup> = w<sup>T</sup>Σw

# Quadratic Programming
It is still a constrainted optimization problem, but now the objective has become <b>quadratic</b>.
### Constraints
> max<sub>w</sub> μ<sup>T</sup>W,

> Subject to μ<sup>T</sup>W = r

> and 1<sub>D</sub><sup>T</sup>w = 1

### Or Other Constraints, for example,

> w<sub>i</sub> ≥ 0 (value of weight i must be greater than or equal to 0)

> w<sub>i</sub> ≤ .5 (value of weight i must be smaller than or equal to 0.5)

### Since <code>scipy.optimize</code> do not have a <code>quadprog()</code> fucntion, we have to use <code>minimize()</code> instead

In [7]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import scipy

In [3]:
names = ['GOOG', 'SBUX', 'KSS', 'NEM']

#Get the stock quote
returns = pd.DataFrame()
for n in names:
    df = pdr.DataReader(n, data_source='yahoo', start='2019-01-01', end='2022-01-01')
    ret = df['Close'].pct_change(1).dropna()
    returns.loc[:, n + '_ret'] = ret

returns

,GOOG_ret,SBUX_ret,KSS_ret,NEM_ret
Date,,,,
2019-01-02,0.009888,-0.001242,0.022008,-0.004906
2019-01-03,-0.028484,-0.043377,-0.021091,-0.002900
2019-01-04,0.053786,0.033155,0.000603,0.000000
2019-01-07,-0.002167,0.000000,0.024093,-0.011926
2019-01-08,0.007385,0.000787,0.017203,0.002944
...,...,...,...,...
2021-12-27,0.006263,0.016463,0.012154,0.010919
2021-12-28,-0.010914,0.011819,-0.031496,0.001828
2021-12-29,0.000386,0.007009,0.017886,0.005474


In [5]:
# Mean return of each stock, also the coefficients of the linear objective function to be minimized.
mean_return = returns.mean()

# D = Number of asset in porfolio
D = len(mean_return) 
D

4

In [ ]:
# Function to minimize
def get_variance(w): 
    return w.dot(cov).dot(w)

# Initial guess
x0 = np.ones(D) / D 

# Optimization Algorithm, SLSQP stance for Sequential Least Square Quadratic Programming
method = 'SLSQP', 

# Upper and Lower Bound for each weight 
bounds = [(0, 0.3)] * D

# Contraints, can either be a dictionary or a list of dictionaries where each dictionary represent a constraint
# 'fun': Function that returns constraint met or not
constraints=[{
        'type': 'eq',
        'fun': lambda w: w.sum() - 1
    }]

scipy.optimize.minimize(
    fun=get_variance, 
    x0=x0, 
    method=method, 
    bounds=bounds, 
    constraints=constraints
)